# Tool Calling Use Cases

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [14]:
import sys
!{sys.executable} -m pip install tavily-python

  Using cached tavily_python-0.5.0-py3-none-any.whl.metadata (11 kB)
Using cached tavily_python-0.5.0-py3-none-any.whl (14 kB)


In [1]:
import warnings
warnings.filterwarnings('ignore')

## Get API keys

In [40]:
from utils import get_tavily_api_key
from utils import get_wolfram_alpha_api_key

TAVILY_API_KEY = get_tavily_api_key()
WOLFRAM_ALPHA_KEY = get_wolfram_alpha_api_key()
#print(WOLFRAM_ALPHA_KEY)

In [31]:
import os
from utils import load_env
load_env()

# Replace 'YOUR_ENV_VAR' with the name of your environment variable
env_var_value = os.getenv('WOLFRAM_ALPHA_KEY')

if env_var_value is not None:
    #print(f"Environment variable value: {env_var_value}")
    print(f"Environment variable value IS SET!!!")
else:
    print("Environment variable is not set.")

Environment variable value IS SET!!!


## Load helper functions

In [3]:
from utils import llama31
from utils import cprint
import json

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Define tool system prompt

In [4]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

17 October 2024


In [5]:
tool_system_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

## The brave_search built-in tool

In [6]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [7]:
response = llama31(prompt)
print(response)



brave_search.call(query="Menlo Park, California weather")


In [8]:
no_tool_call_prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the population of California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

In [9]:
no_tool_call_response = llama31(no_tool_call_prompt)
print(no_tool_call_response)



As of my cut-off knowledge date of 01 March 2023, the estimated population of California is approximately 39.5 million people, according to the United States Census Bureau. However, please note that this information may have changed since my cut-off date. For the most up-to-date information, I recommend checking the latest data from the United States Census Bureau or other reliable sources.


### Calling the search API

In [16]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

result = tavily_client.search("current weather in Menlo Park, California")
cprint(result)

{
    "query": "current weather in Menlo Park, California",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Menlo Park, California",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729134537, 'localtime': '2024-10-16 20:08'}, 'current': {'last_updated_epoch': 1729134000, 'last_updated': '2024-10-16 20:00', 'temp_c': 17.0, 'temp_f': 62.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 3.1, 'wind_kph': 5.0, 'wind_degree': 247, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.93, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 0, 'feelslike_c': 17.0, 'feelslike_f': 62.6, 'windchill_c': 14.7, 'w

### Reprompting Llama with search tool response

In [17]:
search_result = result["results"][0]["content"]
print(search_result)

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729134537, 'localtime': '2024-10-16 20:08'}, 'current': {'last_updated_epoch': 1729134000, 'last_updated': '2024-10-16 20:00', 'temp_c': 17.0, 'temp_f': 62.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 3.1, 'wind_kph': 5.0, 'wind_degree': 247, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.93, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 72, 'cloud': 0, 'feelslike_c': 17.0, 'feelslike_f': 62.6, 'windchill_c': 14.7, 'windchill_f': 58.5, 'heatindex_c': 15.1, 'heatindex_f': 59.2, 'dewpoint_c': 12.1, 'dewpoint_f': 53.8, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 5.0, 'gust_kph': 8.1}}


In [18]:
prompt = tool_system_prompt + f"""<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>{response}<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{search_result}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
print(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 17 October 2024
<|eot_id|><|start_header_id|>user<|end_header_id|>
What is the current weather in Menlo Park, California?

<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|python_tag|>

brave_search.call(query="Menlo Park, California weather")<|eom_id|>
<|start_header_id|>ipython<|end_header_id|>

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1729134537, 'localtime': '2024-10-16 20:08'}, 'current': {'last_updated_epoch': 1729134000, 'last_updated': '2024-10-16 20:00', 'temp_c': 17.0, 'temp_f': 62.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 3.1, 'wind_kph': 5.0, 'wind_degree': 247, 'wind_d

In [19]:
response = llama31(prompt)
print(response)

The current weather in Menlo Park, California is clear with a temperature of 62.6°F (17.0°C) and a wind speed of 5 mph (8 km/h) from the WSW. The humidity is 72% and the pressure is 1014 mb. There is no precipitation expected.


## Using the higher-level message

In [20]:
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [21]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Menlo Park, California?"}
  ]

In [22]:
response = llama31(messages)
print(response)

brave_search.call(query="current weather in Menlo Park, California")


In [23]:
messages = [
    {"role": "system",     "content":  system_prompt_content},
    {"role": "user",       "content": "What is the current weather in Menlo Park, California?"},
    {"role": "assistant",  "content": response},
    {"role": "ipython",    "content": search_result}
  ]

In [24]:
response = llama31(messages)
print(response)

The current weather in Menlo Park, California is clear with a temperature of 62.6°F (17.0°C). The wind is blowing at 5.0 mph (8.1 kph) from the WSW, and the humidity is 72%.


## The Wolfram Alpha tool

In [25]:
math_problem = "Can you help me solve this equation: x^3 - 2x^2 - x + 2 = 0?"

In [26]:
messages = [
    {"role": "system",  "content": system_prompt_content},
    {"role": "user",    "content": math_problem}
]

In [27]:
response = llama31(messages)
print(response)

wolfram_alpha.call(query="solve x^3 - 2x^2 - x + 2 = 0")


### Calling the Wolfram Alpha tool

In [41]:
from wolframalpha import Client

WOLFRAM_ALPHA_KEY = os.getenv('WOLFRAM_ALPHA_KEY')
client = Client(WOLFRAM_ALPHA_KEY)

result = client.query("What is the capital of France?")
print(result)

{'@success': True, '@error': 'false', '@xml:space': 'preserve', '@numpods': '18', '@datatypes': 'AdministrativeDivision,City,Country,MetropolitanArea,NuclearReactor,Person,WeatherStation', '@timedout': '', '@timedoutpods': '', '@timing': '4.382', '@parsetiming': '0.28500000000000003', '@parsetimedout': 'false', '@recalculate': '', '@id': 'MSP47514g3de7564803gg200005518f91de8fc8628', '@host': 'https://www6b3.wolframalpha.com', '@server': '11', '@related': 'https://www6b3.wolframalpha.com/api/v1/relatedQueries.jsp?id=MSPa47614g3de7564803gg2000039e85g2cfgic376a5481992349747128482', '@version': '2.6', '@inputstring': 'What is the capital of France', 'pod': [{'@title': 'Input interpretation', '@scanner': 'Identity', '@id': 'Input', '@position': 100.0, '@error': 'false', '@numsubpods': 1, 'subpod': {'@title': '', 'img': {'@src': 'https://www6b3.wolframalpha.com/Calculate/MSP/MSP47714g3de7564803gg2000055635e2582i7i9if?MSPStoreType=image/gif&s=11', '@alt': 'France | capital city', '@title': 'F

In [37]:
from utils import wolfram_alpha
tool_result = wolfram_alpha("solve x^3 - 2x^2 - x + 2 = 0")
print(tool_result)

x = ± 1
x = 2


### Checking the result

In [42]:
from sympy import symbols, Eq, solve

x = symbols('x')                           # Define the variable
equation = Eq(x**3 - 2*x**2 - 1*x + 2, 0) # Define the equation
solution = solve(equation, x)              # Solve the equation

print(solution)

[-1, 1, 2]


### Reprompting Llama with Wolfram Alpha tool response

In [43]:
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user",      "content": math_problem},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": tool_result}
]

In [44]:
response = llama31(messages)
print(response)

The solutions to the equation x^3 - 2x^2 - x + 2 = 0 are x = ±1 and x = 2.


## The code interpreter built-in tool

In [45]:
loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [46]:
messages = [
    {"role": "system",     "content": system_prompt_content},
    {"role": "user",       "content": loan_question},
  ]

In [47]:
response = llama31(messages)
print(response)

# Define the variables
loan_amount = 1000000  # $1M
interest_rate = 0.06  # 6%
down_payment = 0.20  # 20%
loan_term = 30  # 30 years

# Calculate the down payment
down_payment_amount = loan_amount * down_payment

# Calculate the loan amount after down payment
loan_amount_after_down_payment = loan_amount - down_payment_amount

# Calculate the monthly interest rate
monthly_interest_rate = interest_rate / 12

# Calculate the number of payments
number_of_payments = loan_term * 12

# Calculate the monthly payment
monthly_payment = round(loan_amount_after_down_payment * (monthly_interest_rate * (1 + monthly_interest_rate) ** number_of_payments) / ((1 + monthly_interest_rate) ** number_of_payments - 1), 2)

# Calculate the total payment
total_payment = round(monthly_payment * number_of_payments, 2)

# Calculate the total interest paid
total_interest_paid = round(total_payment - loan_amount_after_down_payment * number_of_payments, 2)

print(f"Monthly payment: ${monthly_payment}")
print(f"Total

# Calculate loan payments

In [48]:
from utils import calculate_loan
monthly_payment, total_payment, total_interest_paid = calculate_loan(
    loan_amount = 1000000, 
    annual_interest_rate = 0.06, 
    loan_term = 30, 
    down_payment = 200000)

print(f"Monthly payment: ${(monthly_payment)}")
print(f"Total payment: ${(total_payment)}")
print(f"Total interest paid: ${(total_interest_paid)}")

Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705


## Generating the code in Java

In [49]:
messages = [
    {"role":    "system",     
     "content": system_prompt_content + "\nGenerate the code in Java."},
    {"role":    "user",       "content": loan_question},
  ]

In [50]:
response = llama31(messages)
print(response)

import java.util.Scanner;

public class MortgageCalculator {
    public static void main(String[] args) {
        // Input values
        double principal = 1000000; // $1M
        double downPayment = 0.2 * principal; // 20% down payment
        double loanAmount = principal - downPayment;
        double annualInterestRate = 0.06; // 6% interest rate
        int years = 30; // 30 year mortgage

        // Calculate monthly interest rate
        double monthlyInterestRate = annualInterestRate / 12;

        // Calculate number of payments
        int numberOfPayments = years * 12;

        // Calculate monthly payment
        double monthlyPayment = loanAmount * (monthlyInterestRate * Math.pow(1 + monthlyInterestRate, numberOfPayments)) / (Math.pow(1 + monthlyInterestRate, numberOfPayments) - 1);

        // Calculate total payment
        double totalPayment = monthlyPayment * numberOfPayments;

        // Calculate total interest paid
        double totalInterestPaid = totalPayment -

### Reprompting Llama with Code Interpreter tool response

In [51]:
code_interpreter_tool_response="""
Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705
"""

In [52]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",      "content": loan_question},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": code_interpreter_tool_response}
  ]

In [53]:
response = llama31(messages)
print(response)

The monthly payment for a 30 year mortgage of $1M at a fixed rate of 6% with a 20% down payment is $4,796. The total payment over the life of the loan is $1,726,705, and the total interest paid is $926,705.


## Llama 3.1 custom tool calling

In [54]:
from utils import trending_songs, get_boiling_point

country = "US"
top_num = 5
top_songs = trending_songs(country, top_num)
print(f"Top {top_num} trending songs in {country}:")
print(top_songs)

Top 5 trending songs in US:
['Blinding Lights - The Weeknd', 'Levitating - Dua Lipa', 'Peaches - Justin Bieber', 'Save Your Tears - The Weeknd', 'Good 4 U - Olivia Rodrigo']


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>trending_songs()</code> in <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Prompt Llama 3.1 for custom tool call

In [55]:
user_prompt = """
Answer the user's question by using the following functions if needed.
If none of the functions can be used, please say so.
Functions (in JSON format):
{
    "type": "function", "function": {
        "name": "get_boiling_point",
        "description": "Get the boiling point of a liquid",
        "parameters": {
            "type": "object", "properties": [
                {"liquid_name": {"type": "object", "description": "name of the liquid"}},
                {"celsius": {"type": "object", "description": "whether to use celsius"}}
            ], "required": ["liquid_name"]
        }
    }
}
{
    "type": "function", "function": {
        "name": "trending_songs",
        "description": "Returns the trending songs on a Music site",
        "parameters": {
            "type": "object", "properties": [
                {"country": {"type": "object", "description": "country to return trending songs for"}},
                {"n": {"type": "object", "description": "The number of songs to return"}}
            ], "required": ["country"]
        }
    }
}

Question: Can you check the top 5 trending songs in US?
"""

In [56]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt}
  ]

In [57]:
result = llama31(messages,405)
print(result)

{"type": "function", "name": "trending_songs", "parameters": {"country": "US", "n": "5"}}


### Calling the custom tool

In [58]:
custom_tools = {"trending_songs": trending_songs,
                "get_boiling_point": get_boiling_point}

In [59]:
res = json.loads(result)
function_name = res['name']
parameters = list(res['parameters'].values())
function_name, parameters

('trending_songs', ['US', '5'])

In [60]:
tool_result = custom_tools[function_name](*parameters)
tool_result

['Blinding Lights - The Weeknd',
 'Levitating - Dua Lipa',
 'Peaches - Justin Bieber',
 'Save Your Tears - The Weeknd',
 'Good 4 U - Olivia Rodrigo']

### Reprompting Llama with custom tool call result

In [61]:
messages = [
    {
      "role": "system", "content":  f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""},
    {"role": "user", "content": user_prompt},
    {"role": "assistant", "content": result},
    {"role": "ipython", "content": ','.join(tool_result)}
  ]

In [62]:
response = llama31(messages, 70)
print(response)

The top 5 trending songs in the US are:

1. Blinding Lights - The Weeknd
2. Levitating - Dua Lipa
3. Peaches - Justin Bieber
4. Save Your Tears - The Weeknd
5. Good 4 U - Olivia Rodrigo
